In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess_(df):
    df = df.iloc[0:,1:]
    X = df.drop(['length','label'], axis = 1)
    y = df['label']

    df.label = pd.Categorical(df.label)
    y = np.array(df.label.cat.codes)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    return X, y

In [9]:
df = pd.read_csv('/home/khangpt/MUSIC-GEN-PROJ/GTZAN/Data/features_30_sec.csv')

In [12]:
X, y = preprocess_(df)

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier,VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


svc = SVC(probability=True)
tree = DecisionTreeClassifier()

# svc.fit(X_train, y_train)
# tree.fit(X_train, y_train)

ada = AdaBoostClassifier()
ada.estimators_ = [svc, tree]
ada.fit(X_train, y_train)

/home/khangpt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier()

In [37]:
from sklearn.metrics import accuracy_score
def assess_model(model):
    pred = model.predict(X_test)
    return accuracy_score(pred, y_test)

In [42]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

class CustomAdaBoostClassifier(BaseEstimator, ClassifierMixin):
  """
  Custom AdaBoostClassifier that allows for different base learners.
  """

  def __init__(self, base_estimator, n_estimators=50, learning_rate=1.0):
    """
    Initialize the AdaBoostClassifier.

    Args:
      base_estimator: The base classifier to use.
      n_estimators (int, optional): The number of boosting stages to perform. Defaults to 50.
      learning_rate (float, optional): The learning rate for each boosting stage. Defaults to 1.0.
    """
    self.base_estimator = base_estimator
    self.n_estimators = n_estimators
    self.learning_rate = learning_rate

  def fit(self, X, y):
    """
    Fit the model to the data.

    Args:
      X: The training data.
      y: The training labels.
    """

    # Initialize weights for each data point (uniform initially)
    self.weights_ = np.ones(len(y)) / len(y)

    estimators_ = []
    for _ in range(self.n_estimators):
      # Fit a base classifier on weighted data
      estimator = self.base_estimator.fit(X, y, sample_weight=self.weights_)
      estimators_.append(estimator)

      # Calculate predictions and errors
      y_pred = estimator.predict(X)
      errors = np.not_equal(y_pred, y)

      # Update weights based on errors
      self.weights_[errors] *= self.learning_rate
      self.weights_ /= np.sum(self.weights_)

    self.estimators_ = estimators_
    return self

  def predict(self, X):
    """
    Predict labels for new data.

    Args:
      X: The data to predict on.

    Returns:
      The predicted labels.
    """

    predictions = np.zeros((X.shape[0], len(self.estimators_)))
    for i, estimator in enumerate(self.estimators_):
      predictions[:, i] = estimator.predict(X)

    # Take a weighted majority vote for final prediction
    final_pred = np.argmax(np.average(predictions, axis=1, weights=self.weights_), axis=1)
    return final_pred

# Example usage
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Define base learners
svc_learner = SVC(probability=True)
tree_learner = DecisionTreeClassifier()

# Create custom AdaBoostClassifier
custom_ada = CustomAdaBoostClassifier(base_estimator=svc_learner, n_estimators=100)

# Fit the model on your training data (X_train, y_train)
custom_ada.fit(X_train, y_train)

# Make predictions on new data (X_test)
y_pred = custom_ada.predict(X_test)


ValueError: Length of weights not compatible with specified axis.